In [1]:
import pandas as pd
import numpy as np

In [2]:
#import data (duplicate Tweets already removed using Twarc)

df = pd.read_csv("data/navalny.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85382 entries, 0 to 85381
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            85382 non-null  int64  
 1   tweet_url                     85382 non-null  object 
 2   created_at                    85382 non-null  object 
 3   parsed_created_at             85382 non-null  object 
 4   user_screen_name              85382 non-null  object 
 5   text                          85382 non-null  object 
 6   tweet_type                    85382 non-null  object 
 7   coordinates                   3 non-null      object 
 8   hashtags                      21391 non-null  object 
 9   media                         7274 non-null   object 
 10  urls                          12356 non-null  object 
 11  favorite_count                85382 non-null  int64  
 12  in_reply_to_screen_name       10206 non-null  object 
 13  i

In [4]:
# tweets are in 51 different languages

# I'll be working only with Tweets in English
# drop tweets in all other languages
# now working with 38884 Tweets 

df = df[df.lang == 'en']
df.shape

# after non-English removed, just 58,071 Tweets left 

(58071, 37)

In [5]:
# drop unnecessary columns 

df = df.drop(['tweet_url', 'created_at', 'media', 'urls','in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'retweet_or_quote_id',
       'retweet_or_quote_screen_name', 'retweet_or_quote_user_id', 'source',
       'user_created_at', 'user_name', 'user_verified', 'user_friends_count', 'user_listed_count',
       'user_statuses_count', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'coordinates', 'lang', 'user_location', 'user_time_zone', 'user_urls', 'place'], axis=1)

In [6]:
# drop retweets

df = df[df.tweet_type != 'retweet']

In [7]:
df.info()

# removing retweets reduces data set to 9294 Tweets

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9294 entries, 4 to 85364
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  9294 non-null   int64 
 1   parsed_created_at   9294 non-null   object
 2   user_screen_name    9294 non-null   object
 3   text                9294 non-null   object
 4   tweet_type          9294 non-null   object
 5   hashtags            1586 non-null   object
 6   favorite_count      9294 non-null   int64 
 7   possibly_sensitive  4033 non-null   object
 8   retweet_count       9294 non-null   int64 
 9   user_id             9294 non-null   int64 
dtypes: int64(4), object(6)
memory usage: 798.7+ KB


In [8]:
# check end time & date of data

df.iloc[0]

# last Tweet downloaded 2021-01-25 02:28:46+00:00

id                                             1353530218187776000
parsed_created_at                        2021-01-25 02:28:46+00:00
user_screen_name                                    EthanMillenium
text                  @teamnavalny @navalny Checkout that ice rink
tweet_type                                                   reply
hashtags                                                       NaN
favorite_count                                                   0
possibly_sensitive                                             NaN
retweet_count                                                    0
user_id                                        1074657406511169536
Name: 4, dtype: object

In [10]:
# check start time & date of data 

df.iloc[-1]

# first Tweet on  2021-01-24 06:02:53+00:00

id                                                  1353221717569773568
parsed_created_at                             2021-01-24 06:02:53+00:00
user_screen_name                                          serozhafromru
text                  @McFaul @navalny Putin doesn't want to move fr...
tweet_type                                                        reply
hashtags                                                            NaN
favorite_count                                                        0
possibly_sensitive                                                  NaN
retweet_count                                                         0
user_id                                                       289723038
Name: 85364, dtype: object

In [11]:
# create variable for "text" column 

text = df['text'] 

In [12]:
# tokenize, remove stopwords, remove urls, lowercase, remove punctuation, remove numbers

# import necessary libraries: ntlk etc.

import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 


stop = stopwords.words('english')

punc = list(set(string.punctuation))

def tokenizer(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

def remove_url(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r'', text)

def process_text(text):
    text = remove_url(text)
    text = tokenizer(text)
    text = [word.lower() for word in text]
    text = [re.sub('[0-9]+', '', word) for word in text]
    text = [word for word in text if word not in punc]
    text = [word for word in text if word not in stop]
    text = [each for each in text if len(each) > 1]
    text = [word for word in text if ' ' not in word]
     
    return text

In [13]:
# apply text processing functions to text

df['processed_text'] = df['text'].apply(process_text)

In [14]:
# look at some of processed text

pd.set_option('display.max_colwidth', -1)
df['processed_text'][:20]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


4      [@teamnavalny, @navalny, checkout, ice, rink]                                                                                                                                                                                                              
8      [alexei, navalny, russian, russian, ever, lived, utterly, unafraid]                                                                                                                                                                                        
23     [@bigbrat, @meghanmccain, @navalny, dad, physically, hand, delivered, fake, hillary, clinton, paid, dossier, lindsey, graham, kicked, entire, russia, charade, key, player, shit, show, republic, may, patriotic, soldier, mccain, civil, servant, traitor]
35     [residents, russia, hold, demonstration, charlottetown, navalny's, arrest, guardian]                                                                                                                                    

In [15]:
# part-of-speech tagging 

ready_for_pos = df['processed_text']

def pos_tagging(text):
    pos_tag = [pos_tag(word) for word in ready_for_pos]

df['pos_tagged'] = df.processed_text.apply(lambda x: pos_tag(x))

In [16]:
# lemmatizing

pos_tagged = df['pos_tagged']

wordnet = WordNetLemmatizer() 

lemmatized = [[wordnet.lemmatize(word[0]) for word in words] for words in pos_tagged]

In [17]:
# look at lemmatized text

df['lemmatized'] = lemmatized
lemmatized[:20]

[['@teamnavalny', '@navalny', 'checkout', 'ice', 'rink'],
 ['alexei',
  'navalny',
  'russian',
  'russian',
  'ever',
  'lived',
  'utterly',
  'unafraid'],
 ['@bigbrat',
  '@meghanmccain',
  '@navalny',
  'dad',
  'physically',
  'hand',
  'delivered',
  'fake',
  'hillary',
  'clinton',
  'paid',
  'dossier',
  'lindsey',
  'graham',
  'kicked',
  'entire',
  'russia',
  'charade',
  'key',
  'player',
  'shit',
  'show',
  'republic',
  'may',
  'patriotic',
  'soldier',
  'mccain',
  'civil',
  'servant',
  'traitor'],
 ['resident',
  'russia',
  'hold',
  'demonstration',
  'charlottetown',
  "navalny's",
  'arrest',
  'guardian'],
 ['get', 'boot', 'yall', 'kush', 'navalny', 'lover'],
 ['terror',
  'authoritarian',
  'government',
  'putin',
  'arrest',
  'thousand',
  'protester',
  'supporting',
  'opposition',
  'leader',
  'navalny',
  'putin',
  'unsuccessfully',
  'poisoned',
  'capitol',
  'rioter',
  'succeeded',
  'jan',
  'country',
  'could',
  'looked',
  'like',
  's

In [18]:
# before vectorizing, cast lists of words back into strings

df['final_docs'] = df['lemmatized'].apply(lambda x: " ".join(x))
pd.set_option('display.max_colwidth', -1)
final_docs = df['final_docs']
final_docs[3000:3020]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


24221    alexei navalny fighting corrupt election trump fighting                                                                                                                                         
24222    @revmagdalen @dmitrydoreshkin @kilroyjoe navalny poisoned dont think russian would dead simple usually happen fall window add sergei skripal case look insidious lived                          
24226    fair point support navalny agree political view russian right political pluralism corruption-free political system                                                                              
24228    @loucharbon @hrw @navalny @nytimes anyone else incredibly proud protestors                                                                                                                      
24240    latest news russia putin face rising discontent amid alexei navalny protest wall street journal #theusaposts #news                                                                     

In [19]:
#create document term matrix with TFIDF

#import vectorizing tool (usee TFIDF)
from sklearn.feature_extraction.text import TfidfVectorizer
# set max_features to 2000 (specifies the number of most frequently occurring words for which we want to create feature vectors)
# set min_df to 5 (word must occur in at least 5 documents)
# set max_df to 0.85 (word must not occur in more than 85 percent of the documents) 

tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.85, ngram_range=(1, 2), stop_words='english')  
doc_term_matrix_1 = tfidfconverter.fit_transform(df['final_docs'].values.astype('U'))

In [20]:
#run NMF model 

#import NMF tool 
from sklearn.decomposition import NMF

nmf_model = NMF(n_components=6)
nmf_Z = nmf_model.fit_transform(doc_term_matrix_1)

In [27]:
# run LDA model

#import LDA tool 
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components = 5, max_iter=10, learning_method='online', learning_decay=.7)
lda_Z = lda_model.fit_transform(doc_term_matrix_1)

In [28]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, tfidfconverter )
print("=" * 30)
 
print("NMF Model:")
print_topics(nmf_model, tfidfconverter )
print("=" * 30)

LDA Model:
Topic 0:
[('alexei', 160.0824794473528), ('alexei navalny', 159.25156436725305), ('russia', 141.7432976865553), ('protest', 129.5625028639081), ('russian', 104.16354899840047), ('support', 101.36902874478842), ('arrested', 94.14136182821055), ('opposition', 93.45675663438386), ('people', 81.99924821705493), ('putin', 77.91231800868768)]
Topic 1:
[('marisepayne', 43.01468946641939), ('marisepayne navalny', 41.59048524236751), ('russian', 40.89159524082136), ('assange', 40.704903289256286), ('russia', 40.190961542143754), ('eu', 39.1050425601515), ('racist', 38.09542235397977), ('nationalist', 35.30052166150479), ('navalny arrest', 35.016031371645035), ('right', 34.78148006113376)]
Topic 2:
[('mcfaul', 115.6076143945626), ('putin', 94.76880007758024), ('mcfaul navalny', 86.72470768257675), ('palace', 55.585825630781635), ('video', 51.33511654198279), ('supporter', 49.90312402873531), ('alexey', 47.78671536274178), ('alexey navalny', 43.785898310162544), ('guy', 41.600344650573

In [29]:
# visualization of LDA model 
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, doc_term_matrix_1, tfidfconverter, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      150.222153 -72.269554   1       1        27.713315
0      51.163364  -159.968018  2       1        26.234610
1     -73.144554   21.523815   3       1        16.081219
2     -66.410133  -110.606438  4       1        15.067317
4      54.417324   12.443444   5       1        14.903539, topic_info=                Term        Freq       Total Category  logprob  loglift
960   mcfaul          113.000000  113.000000  Default  30.0000  30.0000
48    alexei          141.000000  141.000000  Default  29.0000  29.0000
49    alexei navalny  140.000000  140.000000  Default  28.0000  28.0000
1835  trump           191.000000  191.000000  Default  27.0000  27.0000
961   mcfaul navalny  85.000000   85.000000   Default  26.0000  26.0000
...              ...        ...         ...       ...      ...      ...
1722  street          25.009025   46.210102   Topic5  -5.1254   1.2896 
1737  support         27.688610   129.223828  Topic5  -5.0236   0.3630 
1400  putin           27.695842   327.767577  Topic5  -5.0234  -0.5675 
1347  problem         22.744666   38.706336   Topic5  -5.2203   1.3719 
1375  protest russia  22.963372   56.628670   Topic5  -5.2108   1.0010 

[277 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
12    1      0.966315  accountable
16    2      0.987892  act        
28    3      0.933115  affair     
36    1      0.951532  agent      
48    2      0.992357  alexei     
...  ..           ...    ...      
1987  3      0.566286  yes        
1988  2      0.449413  yesterday  
1988  3      0.528722  yesterday  
1994  2      0.244421  youtube    
1994  4      0.733264  youtube    

[361 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3, 5])

In [25]:
# cross-validation to find best parameters for LDA model

#import cross-validation tool
from sklearn.model_selection import GridSearchCV

# define search parameters
search_params = {'n_components': [5, 8, 10, 12], 'learning_decay': [.5, .7, .9]}

# initialize model for cross-validation
lda = LatentDirichletAllocation()

# initialize grid search class 
model = GridSearchCV(lda, param_grid=search_params)

# run grid search 
model.fit(doc_term_matrix_1)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [5, 8, 10, 12]})

In [26]:
# what is best model? 
best_lda_model = model.best_estimator_

# print parameters for best model
print("Parameters for best model: ", model.best_params_)

# print log likelihood score
print("Log likelihood score of best model: ", model.best_score_)

# print perplexity score
print("Perplexity score of best model: ", best_lda_model.perplexity(doc_term_matrix_1))

Parameters for best model:  {'learning_decay': 0.7, 'n_components': 5}
Log likelihood score of best model:  -51445.88317500125
Perplexity score of best model:  2405.3411475214234


In [30]:
# make results of LDA model a column in dataframe

topic_values = lda_model.transform(doc_term_matrix_1)
df['LDA_topic'] = topic_values.argmax(axis=1)

In [60]:
df['text'][df.LDA_topic ==0][1345:1375]

44778    An unusual scene from #Russia where people don't usually fight police at protests. A lone protestor takes on half a dozen riot cops who were unable to arrest him at the time. #navalnyprotests #Navalny https://t.co/oc0DyiATUR                                                                                        
44961    Such #Bravery, #courage&gt;“Tens of thousands of people rallied across #Russia on Saturday in some of the largest demonstrations held against President Vladimir #Putin *in years*. #Navalny https://t.co/1EAhfII1YA                                                                                                    
45020    @RepAdamSchiff @navalny I also stand with them. The entire world will celebrate soon their freedom!!!                                                                                                                                                                                                                   
45046    Massive Protests Sweep Ac

In [34]:
df['text'][df.LDA_topic ==4][40:60]

2746    @TheGhostlyJFK @tonyblunt12 @Brandyski2121 @markmackinnon @bluedillygal @navalny Anytime someone invokes “the deep state” it just means they haven’t got a clue. Either that, or the deep Russian state put them up to it.                                                                             
2758    @WalshFreedom @ColMorrisDavis Navalny is now internationally supported due to his fight against the Putin led government. His struggle is a tough one.\n\nHowever, don't forget about Navalnys nationalist and staunch anti-immigrant background.\n\nA thorough thread on him:\nhttps://t.co/hVBDnYdb2y
2775    @johncusack Mitt has never had the love for Putin, so he's probably glad Navalny is standing up to trump's former lover.                                                                                                                                                                               
2839    @v_zilber @MarkAmesExiled Way better than what it would be under neo-nazi Navaln

In [35]:
df['text'][df.LDA_topic ==1][40:60] # topic 1 references to controversial aspects of Navalny? 

3909    @MaZhavorsiAnni @SamHarrisOrg @CHSommers No, Navalny is making a very specific point about social media. Gingrich never mentioned 'bad tendencies'. He compared an opposition leader who has been arbitrarily locked up and had assassination attempts to a former President who abused his power repeatedly.
3962    Alexei Navalny was not a hero. He was a petty thief, a fraudster, and a US bought spy. Also, a Russian supremacist.                                                                                                                                                                                          
4097    @afrosabi Navalny being racist doesn’t mean Putin isn’t racist. They both are racist. https://t.co/UeqTNvDgMP                                                                                                                                                                                                
4254    @mdunjic @ScepticalAussie @johnpilger The Sceptical Aussie blo

In [68]:
df['text'][df.LDA_topic ==1][1000:1050] # topic 1 references to controversial aspects of Navalny? 

67492    @MarisePayne @navalny How concerned is Australia about Julian Assange?                                                                                                                                                                                                                                                                                                                                                                                        
67511    @TEK10 @McFaul @tedcruz Navalny calls on the West to adopt sanctions against Russia.\nWhat will be done to your politician, who will call for sanctions against the United States?                                                                                                                                                                                                                                                                            
67539    Backed by the Globohomo Mafia and the CIA, #Navalny the oppotunistic snake, sur

In [69]:
df['text'][df.LDA_topic ==1][0:50] # topic 1 references to controversial aspects of Navalny? 

35      P.E.I. residents from Russia hold demonstration in Charlottetown against Navalny's arrest | The Guardian https://t.co/dFrLWAZlPQ                                                                                                                                                                                             
256     EU should punish Putin for Navalny arrest by cutting money flows: Germany's Weber | Reuters https://t.co/gv8XnAh4gg                                                                                                                                                                                                          
296     @POTUS #FreeNavalny seizures of #Putin  &amp; his cronies the banks &amp; money laundering criminals in the USA in place of sanctions on the Russian people to save life of Navalny #https://www.washingtonpost.com/opinions/2021/01/23/biden-must-act-save-navalnys-life-hopes-freedom-russia/                              
310     @DrKevorkO @na

In [70]:
df['text'][df.LDA_topic ==1][200:250] # topic 1 references to controversial aspects of Navalny? 

14763    @jevanaw @MenOfManu @Reuters When Saudi slaughters Yemen, when his prince kills a journalist , no sanction, , but Russia receives sanction because this idiot of Navalny.                                                                                                                                                                                      
14769    @vegetablesgooff @seaniepops707 @MarcDaran @LeethalMe @MarisePayne @navalny Are you saying Assange revealed war crimes? Yes he did and they were horrible. Or are you are saying the UN special Rapporteur uncovered war crimes?                                                                                                                               
14856    @HHankerman @seaniepops707 @MarcDaran @LeethalMe @MarisePayne @navalny You keep going back to this \n\nTrump's DOJ's charges are for revealing war crimes, and you're happy to support Trump in that because you dislike Trump so much &amp; want to punish someone in some s

In [71]:
df['text'][df.LDA_topic ==1][250:300]

17396    @makaroni_i_sir in nazi germany, people also had agency &amp; sometimes did horrible stuff, but when we talk about nazism, do we center on their agency solely or discuss the harm of nazi propaganda, brainwashing, etc.? same here. navalny is propaganda. and this propaganda affects young people                                                         
17454    @IvanKlyuzhin @navalny That is great. Was shocked because I had not read anything.✌️                                                                                                                                                                                                                                                                          
17492    https://t.co/mnMmRRO4z3 Navalny team releases investigation into 'Putin's Palace'                                                                                                                                                                                              

In [46]:
df['text'][df.LDA_topic ==2][900:930] 

65864    Alexei Navalny’s home makeover show exposes Vladimir Putin on every front | Rowan Moore https://t.co/6AxGkRTot6                                                                                                                                                                                                      
66135    @Reuters Navalny is a fucking cunt not a critic                                                                                                                                                                                                                                                                      
66365    @badiucao @navalny @pussyrrriot Thanks for the lovely cartoon. Watch a movie about Putin filmed by Navalny and his FBK Foundation. FBK exists on donations from Russians. This film explains the nature of Putin, the source of his wealth. The movie has subtitles https://t.co/fVYiUWOQvs                          
66416    Navalny reveals investigation into

In [58]:
df['text'][df.LDA_topic ==2][46
                             0:480] 

33939    @ManuelaSchwesig are you a paid agent of Russia? @navalny                                                                                                                                                                                                                                            
34004    Source :\n\nhttps://t.co/Xcup299hjx                                                                                                                                                                                                                                                                  
34252    Alexei Navalny’s home makeover show exposes Vladimir Putin on every front.\nFirst Trump, now Putin. https://t.co/lWoQ54ReVK                                                                                                                                                                          
34258    Police crack down on Russian protests against jailing of Kremlin foe Navalny https

In [42]:
df['text'][df.LDA_topic ==3][700] 

7412    Where is the evidence, @navalny ?\nThe only way to make mountains of money with material that disappears is either that the material never existed or that it existed and was sold twice. https://t.co/EGAmPGznX2                                                                             
7415    @newtgingrich it shows how delusional you are that you think Trump is Navalny - that's the best new year joke. Imprisoning Hitler did not deepen the divide in Germany in 1945 .....                                                                                                          
7416    @RussianEmbassy It was a professionally attempted hit on Navalny.  There, I fixed it for you.  I guess I’l novichok it up to Russian propaganda.                                                                                                                                              
7448    @newtgingrich Shouldn't you be colonizing the moon?🤪🤪🤪\nIt's laughable that you're comparing Putin's former

In [67]:
df['text'][df.LDA_topic ==4][1000:1050] # mixed views on Navalny

82578    Pro-Navalny Protests Sweep Russia in Challenge to Putin https://t.co/Tq0pWjHDVX                                                                                                                                                                                                                                                                          
82715    Russia accuses US diplomats of publishing Navalny rally routes https://t.co/b0l9Ctz84F                                                                                                                                                                                                                                                                   
82716    Russia: Police detain thousands at pro-Navalny protests https://t.co/ihPq990HaR                                                                                                                                                                                                          

In [66]:
df['text'][df.LDA_topic ==4][1000:1050] # mixed views on Navalny

82578    Pro-Navalny Protests Sweep Russia in Challenge to Putin https://t.co/Tq0pWjHDVX                                                                                                                                                                                                                                                                          
82715    Russia accuses US diplomats of publishing Navalny rally routes https://t.co/b0l9Ctz84F                                                                                                                                                                                                                                                                   
82716    Russia: Police detain thousands at pro-Navalny protests https://t.co/ihPq990HaR                                                                                                                                                                                                          

In [75]:
df['text'][df.LDA_topic ==1][400:450]  # topic 1 references to controversial aspects of Navalny? 

26918    https://t.co/di8JUbGvSA\n\nchinese russian alliance. things are moving fast                                                                                                                                                                                                                                                                                            
26982    @brianjtrautman @johncusack @artemiypanarin freedom for Navalny but also freedom from Navalny, Russia deserves better than both him and Putin                                                                                                                                                                                                                          
27073    @LouiseMensch @navalny Pretty bold of them to admit the #Navalny case is fake though. Novel approach.                                                                                                                                                        

In [78]:
df['text'][df.LDA_topic ==1][550:600]  

36365    @aaron_schwa @kvoneggert Regardless of foreign policy, Russia under Navalny would have a dramatically different relationship with western capital, yes? And this, in turn, would change internal power dynamics between various classes/interests?                                                                
36584    @ItsMutai Kremlin is unshakable , Navalny is a West puppet                                                                                                                                                                                                                                                        
36586    The west-supported and celebrated Pashinyan and his team in Armenia has been another failure, both in terms of diminishing liberal democracy and emboldening cultural fascism, as well as destabilizing the entire region. Save the lesson for Russia and Navalny 😉 https://t.co/Bjj3UD2kFa                       
36611    A very good analysis of yesterday’s protest

In [79]:
df['text'][df.LDA_topic ==1][700:750]  

45895    @ABarbashin but, even as navalny himself told der spiegel, there is very little reason for us to view the KPRF as "left" in any real sense                                                                                                                                                                                                                         
45991    @DanRather @SusanDFoster1 And the voice of #Navalny .                                                                                                                                                                                                                                                                                                              
46044    @McFaul - and Navalny is still sitting in prison and posting on instagram ;-)\nApparently the new trend is to give detainees a smartphone so that they can keep up with their social networking ;-)\n\nAnd still alive.  :-) When Magnitsky and Nemtsov and Politkovs

In [80]:
df['text'][df.LDA_topic ==1][750:800]  

49491    @FadilAliriza @RaniaKhalek @MeshkalTn @Ghaya_BM If you say that Putin is maybe certainly behind the protests or the government you will get it. Russiagate Africa. Europeans, especially Germany and France, have the same interests when propagandize Navalny and hide all the popular uprisings happening in Africa: Imperialism XXI.
49509    @p_zalewski @Andrew__Roth Free NAVALNY !!! Thoughts and prayers from USA                                                                                                                                                                                                                                                               
49596    @PostOpinions @MaxBoot @AlinejadMasih As Navalny was saying, tech titans must be consistent on their content policies and have a process. Then they could go after spreaders of hate wherever they are: in Iran, US, Israel, Philippines, Russia... Otherwise it’s just opportunistic censorship.                            